T-1日到T日结算价变化方向  logit回归  T+1日看涨/看跌行为

| 结算价变化 | 变量 |
| ---------- | ---- |
|   上涨   |  +1  |
|   下跌   |  -1  |

|持仓量变化|行为类型|
|----------|-------|
|买持仓增加|看涨|
|买持仓减少|看跌|
|卖持仓增加|看跌|
|卖持仓减少|看涨|

In [ ]:
import MySQLdb
# from sklearn.linear_model import LogisticRegression
from statsmodels.discrete.discrete_model import Logit
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('error')

conn = MySQLdb.connect(host='219.224.169.45', user='lizimeng', passwd='codegeass', db='investor', charset='utf8')
cursor = conn.cursor()

# 账号
accounts = []
sql = 'select distinct capital_account_new from chicang'
cursor.execute(sql)
for row in cursor.fetchall():
    accounts.append(row[0])
    
# 日期 (昨天、今天、明天)
days = []
lastdays = []
nextdays = []
sql = "select day,lastday,nextday from commodity.trade_day where day between '20140101' and '20161231' \
order by day asc"
cursor.execute(sql)
for row in cursor.fetchall()[1:-1]:
    days.append(row[0])
    lastdays.append(row[1])
    nextdays.append(row[2])


for account in accounts:
    X = []
    Y = []
    for i in range(len(days)):
        day = days[i]
        lastday = lastdays[i]
        nextday = nextdays[i]
        #  今日持仓合约 或 明日持仓合约
        sql = "(select distinct vari_code,deliv_date from chicang where capital_account_new=%s and tradedate=%s) union \
        (select distinct vari_code,deliv_date from chicang where capital_account_new=%s and tradedate=%s)"
        cursor.execute(sql, (account,day,account,nextday))
        contracts = cursor.fetchall()
        for contract in contracts:
            # 结算价（要求昨天和今天都有结算价）
            sql = "select settle from commodity.contract_daily where date=%s and code=%s and delivery=%s"
            if cursor.execute(sql, (day,contract[0],contract[1]))==0:
                continue
            settle = cursor.fetchone()[0]
            if cursor.execute(sql, (lastday,contract[0],contract[1]))==0:
                continue
            settle_last = cursor.fetchone()[0]
            if settle==settle_last:
                continue
            x = 1. if settle>settle_last else -1.
            x = (x, 1.)    # 增加截距
            # 持仓
            sql = "select sum(qty) from chicang where capital_account_new=%s and tradedate=%s and vari_code=%s and \
            deliv_date=%s and bs_flag=%s"
            for bs_flag in ('0','1'):
                if cursor.execute(sql, (account,day,contract[0],contract[1],bs_flag))>0:
                    oi = cursor.fetchone()[0]
                else:
                    oi = 0
                if cursor.execute(sql, (account,nextday,contract[0],contract[1],bs_flag))>0:
                    oi_next = cursor.fetchone()[0]
                else:
                    oi_next = 0
                if bs_flag=='0' and oi_next>oi:
                    y = 1
                elif bs_flag=='0' and oi_next<oi:
                    y = 0
                elif bs_flag=='1' and oi_next>oi:
                    y = 0
                elif bs_flag=='1' and oi_next<oi:
                    y = 1
                else:
                    continue
                X.append(x)
                Y.append(y)
    samples = len(X)
    if samples<10:
        continue
    logit = Logit(Y, X)
    try:
        result = logit.fit(maxiter=100, disp=False)
    except:
        continue
    w,b = result.params
    p_w,p_b = result.pvalues
    p1,p2 = result.predict(((1.,1.), (-1.,1.)))
    p2 = 1 - p2
    print account
    print w,b
    print p_w, p_b
    print 1-p1,p1
    print p2,1-p2
    print
    #sql = "insert into price_change_direction2behavior (account,w,b,p_w,p_b,p1,p2,samples) values (" + "%s,"*7 + "%s)"
    #cursor.execute(sql, (account,w,b,p_w,p_b,p1,p2,samples))
    #conn.commit()
    # break

cursor.close()
conn.close()

488854682519
-0.0079276858949 0.019100986493
0.943117740915 0.863504121717
0.497206703911 0.502793296089
0.493243243243 0.506756756757

488854682822
-0.0200668985394 -0.0825872555207
0.905372543409 0.624663931206
0.525641025641 0.474358974359
0.515625 0.484375

488854683120
0.170463293485 -0.0526802578289
0.680640775362 0.898783600551
0.470588235294 0.529411764706
0.555555555556 0.444444444444

488854692522
0.148217431781 0.0230542888267
0.143993325712 0.820220103654
0.457286432161 0.542713567839
0.53125 0.46875

488854693221
0.0161970755611 0.0242942857937
0.85716841402 0.787194894762
0.48987854251 0.51012145749
0.497975708502 0.502024291498

488854702517
-0.501278295225 0.0420945801079
2.82643024841e-11 0.57624444779
0.612820512821 0.387179487179
0.367403314917 0.632596685083

488854702522
-0.155043231059 0.018467696053
0.0954985701014 0.842598185858
0.534090909091 0.465909090909
0.456730769231 0.543269230769

489054682516
0.423648930194 0.135966857742
0.39113076818 0.783139538396
0.

499054683214
-0.676914279935 -0.0409255132152
0.000695398737006 0.837541170197
0.672131147541 0.327868852459
0.346153846154 0.653846153846

499054683314
-0.195538185328 0.0557762429529
0.354139298225 0.791544887826
0.53488372093 0.46511627907
0.4375 0.5625

499054692515
0.12565721414 -0.12565721414
0.622318200599 0.622318200599
0.5 0.5
0.5625 0.4375

499054693021
-0.0136994870941 -0.0136994870941
0.93792349452 0.93792349452
0.506849315068 0.493150684932
0.5 0.5

499054693223
-0.0063186740478 0.0634770878877
0.935454741744 0.415896802392
0.485714285714 0.514285714286
0.482558139535 0.517441860465

499054693316
-0.626381484248 -0.47223080442
0.328374869151 0.461208082262
0.75 0.25
0.461538461538 0.538461538462

499054693323
-0.00587809855883 -0.108532252619
0.97156156853 0.510388556086
0.528571428571 0.471428571429
0.525641025641 0.474358974359

499054693422
0.447835722336 0.091160778397
0.190942249866 0.790075033288
0.368421052632 0.631578947368
0.588235294118 0.411764705882

4990547025

499054753322
0.244176383957 0.0435056884948
0.43799193002 0.890090964361
0.428571428571 0.571428571429
0.55 0.45

499054753414
0.110910181118 0.0751920985159
0.425030277003 0.588630731503
0.453608247423 0.546391752577
0.508928571429 0.491071428571

499054762614
0.279807893968 0.12565721414
0.511376102649 0.768061258395
0.4 0.6
0.538461538462 0.461538461538

499054762715
-0.129302430772 0.0362120077058
0.205811046157 0.723099992853
0.523255813953 0.476744186047
0.45871559633 0.54128440367

499054763122
0.408972446608 -0.0787307597373
0.00464079530207 0.585763678017
0.418181818182 0.581818181818
0.619565217391 0.380434782609

499054763323
-0.21396898198 0.0961859463237
0.249016929069 0.604322955194
0.529411764706 0.470588235294
0.423076923077 0.576923076923

499054763422
0.122707353077 -0.0144937684366
0.457822655111 0.930119822283
0.472972972973 0.527027027027
0.534246575342 0.465753424658

499054772622
0.0706567665333 0.224807446361
0.67874339679 0.187575722625
0.426666666667 0.5733333

499055732617
-0.0232829351836 0.0705358200341
0.852625097889 0.573576743352
0.488188976378 0.511811023622
0.4765625 0.5234375

499055732716
-0.0013593121195 -0.0246161742838
0.99022468021 0.824413270256
0.506493506494 0.493506493506
0.505813953488 0.494186046512

499055732815
-0.0151260215853 0.0404438295696
0.913983567177 0.772726500976
0.493670886076 0.506329113924
0.486111111111 0.513888888889

499055732820
-0.0416908044695 -0.0201845992486
0.77396332451 0.889414401849
0.515463917526 0.484536082474
0.494623655914 0.505376344086

499055733020
0.0667656963123 -0.0667656963123
0.8702066017 0.8702066017
0.5 0.5
0.533333333333 0.466666666667

499055733114
0.0145986174108 -0.00598529973269
0.724763632293 0.885218109393
0.497846683893 0.502153316107
0.505145797599 0.494854202401

499055733115
-0.202732554054 -0.00857653961312
0.404612959347 0.971874649742
0.552631578947 0.447368421053
0.451612903226 0.548387096774

499055733119
0.522454054848 -0.0116284310821
0.0194577256506 0.958522764177